# COVID-19 Data Analysis & Visualization

## What is COVID-19?

>Coronavirus (COVID-19) symptoms are similar to colds and flu. They include a high temperature, a cough and a loss or change to your smell or taste. You can help stop the spread of coronavirus (COVID-19) by getting vaccinated and taking care when meeting other people. While you’re staying at home with coronavirus (COVID-19), you can ease mild symptoms by resting, drinking plenty of fluids and taking painkillers. Treatments for coronavirus (COVID-19) include antibody and antiviral medicines. They are only for people at highest risk of becoming seriously ill.

![Coronavirus Image](https://www.imperial.ac.uk/stories/two-years-of-covid/assets/JeG4hqRj7c/covid-imperial-story-title-image-wide-2560x1440.jpeg)

In [1]:
# Importing the library
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

import plotly.express as px
import plotly.graph_objects as go 

from IPython.core.display import display, HTML
from ipywidgets import interact
from ipywidgets import widgets 
import folium

In [2]:
# loading data right from the source:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

In [3]:
death_df.columns = map(str.lower, death_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
country_df.columns = map(str.lower, country_df.columns)

In [4]:
# Renaming some of the columns for ease of handling
death_df = death_df.rename(columns={'province/state':'state', 'country/region':'country'})
confirmed_df = confirmed_df.rename(columns={'province/state':'state', 'country/region':'country'})
recovered_df= recovered_df.rename(columns={'province/state':'state', 'country/region':'country'})
country_df = country_df.rename(columns={ 'country_region':'country'})

In [5]:
# Creating a new feature "NewCases" to capture the difference between the last/latest day count and 2nd last day count
try:
    confirmed_df.insert(4,'NewCases',0)
except:
    pass
confirmed_df['NewCases'] = confirmed_df.iloc[:,-1] - confirmed_df.iloc[:,-2]

In [6]:
# Creating a new feature "NewCases" to capture the difference between the last/latest day count and 2nd last day count
try:
    death_df.insert(4,'NewCases',0)
except:
    pass
death_df['NewCases'] = death_df.iloc[:,-1] - death_df.iloc[:,-2]

In [7]:
# Creating a new feature "NewCases" to capture the difference between the last/latest day count and 2nd last day count
try:
    recovered_df.insert(4,'NewCases',0)
except:
    pass
recovered_df['NewCases'] = recovered_df.iloc[:,-1] - recovered_df.iloc[:,-2]

In [8]:
# Summing up the total confiremed cases acrons countries
confirmed_total = country_df['confirmed'].sum()


In [9]:
# Summing up the total deaths across countries
deaths_total = country_df['deaths'].sum()


In [10]:
# Summing up the total recovered cases across countries
recovered_total = country_df['recovered'].sum()


In [11]:
# Summing up the total active cases across countries
active_total = country_df['active'].sum()


In [12]:
# display the current total stats

display(
    HTML(
        "<div style = 'background-color: #504e4e; padding:32px '>"+
        "<span style='color: #fff; font-size:32px;'> Confirmed: " + str(confirmed_total) + "</span>" + "<br>" +
        "<span style='color: red; font-size:32px;'> Deaths: " + str(deaths_total) + "</span>" + "<br>" +
        "<span style='color: lightgreen; font-size:32px;'> Recovered: " + str(recovered_total) + "</span>" + "<br>" +
        "<span style='color: lightblue; font-size:32px;'> Active: " + str(active_total) + "</span>" + "<br>" +
        "</div>"
    )
)

### Data Visualization through Bubble Chars

#### Latest Count of Confirmed New Cases

In [13]:
# Aggregation the confirmed new cases againt each country
country_confirmed_newcases = confirmed_df.groupby("country")['NewCases'].sum().reset_index(name = "TotalNewCases")
country_confirmed_newcases = country_confirmed_newcases.sort_values(by='TotalNewCases', ascending=False)


In [14]:
# Visualizing the new confirmed cases against each country using plotly
def bubble_chart(n):
    fig = px.scatter(country_confirmed_newcases.head(n), x="country", y="TotalNewCases", size="TotalNewCases", color="country", 
        hover_name="country", size_max=60
        )
    fig.update_layout(
        title = str(n) + " countries with highest number of confirmed new cases. <br> (Last updated on " + (confirmed_df.columns)[-1] + ")",
        xaxis_title="Countries",
        yaxis_title="Confirmed New Cases",
        width = 900
    )
    fig.show();

interact(bubble_chart, n=10)
ipywLayout= widgets.Layout()
ipywLayout.display='none'


interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

#### Latest Count of New Death Cases

In [15]:
# Aggregating the new death cases against each country
country_death_newcases = death_df.groupby("country")['NewCases'].sum().reset_index(name ='TotalNewCases')
country_death_newcases = country_death_newcases.sort_values(by='TotalNewCases', ascending=False)


In [16]:
# Visualizing the new death cases against each country using plotly
def bubble_chart(n):
    fig = px.scatter(country_death_newcases.head(n), x="country", y="TotalNewCases", size="TotalNewCases", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" countries with highest number of new death cases. <br> (Last updated on "+(death_df.columns)[-1]+")",
    xaxis_title="Countries",
    yaxis_title="New Death Cases",
    width = 900
    )
    fig.show();

    
interact(bubble_chart, n=10)
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

#### Latest Count of New Recovered Cases

In [17]:
# Aggregating the recovered new cases against each country
country_recovered_newcases = recovered_df.groupby("country")['NewCases'].sum().reset_index(name ='TotalNewCases')
country_recovered_newcases = country_recovered_newcases.sort_values(by='TotalNewCases', ascending=False)


In [18]:
# Visualizing the recovered new cases against each country using plotly
def bubble_chart(n):
    fig = px.scatter(country_recovered_newcases.head(n), x="country", y="TotalNewCases", size="TotalNewCases", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" countries with highest number of new recovered cases. <br> (Last updated on "+(recovered_df.columns)[-1]+")",
    xaxis_title="Countries",
    yaxis_title="New Recovered Cases",
    width = 900
    )
    fig.show();

    
interact(bubble_chart, n=10)
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

### Data Visualisation through Line Charts
#### Trend of Confirmed Case

In [19]:
# Visualizing the trend of confirmed cases over time using Plotly
def confirmedCases_trend(name):
    x_data = confirmed_df.iloc[:, 5:].columns
    y_data = confirmed_df[confirmed_df['country'] == name].iloc[:, 5:].sum(axis=0)

    fig = go.Figure()
    fig.add_trace(
        go.Scatter(
            x=x_data, y=y_data, mode='markers', name='markers'
        )
    )

    fig.update_layout(
        title=str(name) + "'s trend on confirmed cases. <br> (Last updated on " + (confirmed_df.columns)[-1]+")",
        xaxis_title="Date",
        yaxis_title="Confirmed Cases",
        width=800
    )

    fig.show()

interact(confirmedCases_trend, name='Nigeria')
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(Text(value='Nigeria', description='name'), Output()), _dom_classes=('widget-interact',))

#### Trend of Death Cases

In [20]:
# Visualizing the trend of death cases over time using Plotly
def deathCases_trend(name):
    x_data = death_df.iloc[:, 5:].columns
    y_data = death_df[death_df['country'] == name].iloc[:,5:].sum(axis=0)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_data, y=y_data,
                    mode='markers',
                    name='markers'))

    fig.update_layout(
        title=str(name) +"'s trend on death cases. <br> (Last updated on "+(death_df.columns)[-1]+")",
        xaxis_title="Date",
        yaxis_title="Death Cases",
        width = 800
        )

    fig.show()
    

interact(deathCases_trend, name='Nigeria')
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(Text(value='Nigeria', description='name'), Output()), _dom_classes=('widget-interact',))

#### Trend of Recovered Case

In [21]:
# Visualizing the trend of recovered cases over time using Plotly
def recoveredCases_trend(name):
    x_data = recovered_df.iloc[:, 5:].columns
    y_data = recovered_df[recovered_df['country'] == name].iloc[:,5:].sum(axis=0)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_data, y=y_data,
                    mode='markers',
                    name='markers'))

    fig.update_layout(
        title=str(name) +"'s trend on recovered casess. <br> (Last updated on "+(recovered_df.columns)[-1]+")",
        xaxis_title="Date",
        yaxis_title="Recovered Cases",
        width = 800
        )

    fig.show()
    
    
interact(recoveredCases_trend, name='Nigeria')
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(Text(value='Nigeria', description='name'), Output()), _dom_classes=('widget-interact',))

### Data Visualization through Bar Charts
#### Worst Hit Countries - Confirmed Cases

In [22]:
def deathCases_bar_chart(n):
    df1 = country_df.sort_values(by='deaths', ascending=False)
    fig = px.bar(df1.head(n), x="country", y="deaths")

    fig.update_layout(
    title=str(n) +" countries with highest number of death cases. <br> (Last updated on "+ df1.last_update[1] +")",
    xaxis_title="Countries",
    yaxis_title="Death Cases",
    width = 800
    )
    
    fig.show();


interact(deathCases_bar_chart, n=10)
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

#### Countries - Recovered Cases

In [23]:
# Visualizing the top n countries with respect to active cases using Plotly
def activeCases_bar_chart(n):
    df1 = country_df.sort_values(by ='active', ascending=False)
    fig = px.bar(df1.head(n), x="country", y="recovered")
    
    fig.update_layout(
    title=str(n) +" countries with highest number of active cases. <br> (Last updated on "+ df1.last_update[1] +")",
    xaxis_title="Countries",
    yaxis_title="Active Cases",
    width = 800
    )
    
    fig.show();

    
interact(activeCases_bar_chart, n=10)
ipywLayout = widgets.Layout()
ipywLayout.display='none'

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

### Data Visualization on Maps
#### Global spread of COVID-19 using Folium

In [24]:
# Removing the null values from confirmed_df
confirmed_df = confirmed_df[~confirmed_df['lat'].isnull()]

In [25]:
# Using Folium to spread of COVID-19 cases over the world
world_map = folium.Map(location=[11,0],  tiles="cartodbpositron", zoom_start=2, max_zoom=6, min_zoom=2)

for i in range(0, len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*5000,
        color='red',
        fill_color='indigo',tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",

    ).add_to(world_map)
world_map

### Global spread of COVID-19 using Plotly

In [26]:
fig = px.scatter_mapbox(confirmed_df, lat="lat", lon="long", color="country",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=0,
                  mapbox_style="carto-positron")

fig.update_layout(
    title="Global spread of COVID-19. (Last updated on "+ confirmed_df.columns[-1] +")"
    )
    
fig.show();